In [ ]:
from pydantic import BaseModel, EmailStr, Field, model_validator, HttpUrl
from typing import Union, Optional, Dict, List
from datetime import datetime, date
from uuid import UUID, uuid4
from enum import Enum
from pydantic_settings import BaseSettings, SettingsConfigDict

In [ ]:
class FirstClass(BaseModel):
    first_name : str
    last_name : str
    
first_object = FirstClass(first_name = 'boiled', last_name = 'potato')
print(first_object)

In [ ]:
class SecondClass(BaseModel):
    first_name : str
    middle_name : Union[str, None] = None 
    title : Optional[str]
    last_name : str

In [ ]:
second_object = SecondClass(first_name = 'boiled', title = '', last_name = 'potato')
print(second_object)

In [ ]:
class ThirdClass(BaseModel):
    names : Dict[str , str]
    skills : List[str]
    holidays : List[Union[str, datetime]]

In [ ]:
third_object = ThirdClass(
    names = {'boiled' : 'potato', 'hello' : 'world', 'thefarm' : 'guy'},
    skills = ['cricket', 'football', 'farming'],
    holidays = [datetime.now(), '10-04-2025', datetime(2025, 4, 10, 14, 30, 0)]
)
print(third_object) 

In [ ]:
class Department(Enum):
    HR = 'HR'
    SALES = 'SALES'
    IT = 'IT'
    ENGINEERING = 'ENGINEERING'
    
class FourthClass(BaseModel):
    employee_id : UUID = uuid4()
    name : str
    email : EmailStr
    date_of_birth : date
    salary : float
    department : Department
    elected_benefits : bool 

In [ ]:
fourth_object = FourthClass(
    # employee_id is default uuid4 so we will skip that because it will be added automatically.
    name = 'boiled potato',
    email = 'boiled@potato.com',
    date_of_birth = '2004-09-14',
    salary = 1000,
    department = 'SALES',
    elected_benefits = False
)

print(fourth_object)
print(fourth_object.department)

In [ ]:
# validate new data according to already built class.
new_employee_json = """
{
    "employee_id" :"d2e7b773-926b-49df-939a-5e98cbb9c9eb",
    "name" :"hello world",
    "email" :"eslogrenta@example.com",
    "date_of_birth" :"1990-01-02",
    "salary" :125000.0,
    "department" :"HR",
    "elected_benefits" :false
}
"""

new_employee = FourthClass.model_validate_json(new_employee_json)
print(new_employee)

In [ ]:
# It returns date differently.
new_employee.model_dump_json()

In [ ]:
fourth_object.model_dump()

In [ ]:
fourth_object.model_json_schema()

In [ ]:
class FifthClass(BaseModel):
    employee_id : UUID = Field(default_factory = uuid4, frozen = True)
    name : str = Field(min_length = 3, frozen = True)
    email : EmailStr = Field(pattern = r".+@example\.com$")
    date_of_birth : date = Field(alias = "birth_date", repr = False, frozen = True)
    salary : float = Field(alias = "compensation", gt = 0, repr = False)
    department : Department
    elected_benefits : bool 

In [ ]:
fifth_object = FifthClass(
    # employee_id is default uuid4 so we will skip that because it will be added automatically.
    name = 'boiled potato',
    email = 'boiled@example.com',  
    birth_date = '2004-09-14',     
    compensation = 1000,          
    department = 'SALES',
    elected_benefits = False
)

fifth_object.model_dump()

In [ ]:
class SixthClass(BaseModel):
    employee_id : UUID = uuid4()
    name : str
    email : EmailStr
    date_of_birth : date
    salary : float
    department : Department
    elected_benefits : bool
    
    @model_validator(mode = "before")
    def lower_name(cls, data):
        data['name'] = data['name'].lower()
        return data
    
    @model_validator(mode = "after")
    def check_age(cls, data):
        date_of_birth = data.date_of_birth
        
        if date_of_birth:
            today = date.today()
            age = today.year - date_of_birth.year - ((today.month, today.day) < (date_of_birth.month, date_of_birth.day))
            
            if age < 18:
                raise ValueError(f'age must be equal or greater than 18, but got {age}')
            
        return data


In [ ]:
fifth_object = SixthClass(
    # employee_id is default uuid4 so we will skip that because it will be added automatically.
    name = 'BOILED POTATO',
    email = 'boiled@potato.com',  
    date_of_birth = '2004-09-14',     
    salary = 1000,          
    department = 'SALES',
    elected_benefits = False
)
fifth_object.model_dump()

In [ ]:
class Settings(BaseSettings):
    database_host : HttpUrl
    database_user : str = Field(min_length = 3)
    database_password : str = Field(min_length = 7)
    api_key : str = Field(min_length = 5)
    
    model_config = SettingsConfigDict(
        env_file = '.env',
        case_sensitive = True,
        extra = 'allow'
    )

In [ ]:
settings = Settings()
settings.model_dump()